# Tradable Price Feature Pipeline Notebook

### Imports

In [1]:
import requests
import pandas as pd
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
import pytz
import numpy as np
warnings.filterwarnings("ignore")

### Login to Hopsworks 


In [2]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
with open('../data/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()

project = hopsworks.login(project = 'StockPrediction', api_key_value=os.environ["HOPSWORKS_API_KEY"])
fs = project.get_feature_store()

api_key_file = '../data/alphavantage-api-key.txt'
util.check_file_path(api_key_file)

with open(api_key_file, 'r') as file:
    ALPHAVANTAGE_API_KEY = file.read().rstrip()

secrets_api = hopsworks.get_secrets_api()
try:
    secrets_api.create_secret('AV_API_KEY', ALPHAVANTAGE_API_KEY)
except hopsworks.RestAPIError:
    ALPHAVANTAGE_API_KEY = secrets_api.get_secret("AV_API_KEY").value


2025-01-06 12:24:11,433 INFO: Initializing external client
2025-01-06 12:24:11,436 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 12:24:13,369 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1205424
File successfully found at the path: ../data/alphavantage-api-key.txt


### Get references to the Feature Groups

In [5]:
# Retrieve feature groups
SPOT_fg = fs.get_feature_group(
    name='spot',
    version=1,
)

GOOGL_fg = fs.get_feature_group(
    name='googl',
    version=1,
)

BTC_fg = fs.get_feature_group(
    name='btc',
    version=1,
)

SandP_fg = fs.get_feature_group(
    name='sandp',
    version=1,
)

Sent_SPOT_fg = fs.get_feature_group(
    name='spotify_sentiment',
    version=1,
)

Sent_GOOGL_fg = fs.get_feature_group(
    name='google_sentiment',
    version=1,
)

Sent_BTC_fg = fs.get_feature_group(
    name='bitcoin_sentiment',
    version=1,
)

### Retrieve Today's Tradable Price data from AV API


In [ ]:
spot_today = util.get_stock_price("SPOT", ALPHAVANTAGE_API_KEY)
googl_today = util.get_stock_price("GOOGL", ALPHAVANTAGE_API_KEY)
btc_today = util.get_crypto_price("BTC", ALPHAVANTAGE_API_KEY)
sandp_today = util.get_stock_price("IVV", ALPHAVANTAGE_API_KEY)
sent_spot_today = util.get_sentiment_score("SPOT", ALPHAVANTAGE_API_KEY)
sent_googl_today = util.get_sentiment_score("GOOG", ALPHAVANTAGE_API_KEY)
sent_btc_today = util.get_sentiment_score("CRYPTO:BTC", ALPHAVANTAGE_API_KEY)

KeyError: 'Meta Data'

### Uploading new data to the Feature Store

In [21]:
# Insert new data

SPOT_fg.insert(spot_today)
GOOGL_fg.insert(googl_today)
BTC_fg.insert(btc_today)
SandP_fg.insert(sandp_today)
Sent_SPOT_fg.insert(sent_spot_today)
Sent_GOOGL_fg.insert(sent_googl_today)
Sent_BTC_fg.insert(sent_btc_today)

2025-01-06 12:07:13,023 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1393604


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: spot_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/spot_1_offline_fg_materialization/executions
2025-01-06 12:07:43,438 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1205424/fs/1194062/fg/1394635


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 01:21 | Remaining Time: 00:00


Launching job: googl_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1205424/jobs/named/googl_1_offline_fg_materialization/executions


FeatureStoreException: Features are not compatible with Feature Group schema: 
 - timestamp (expected type: 'timestamp', derived from input: 'date') has the wrong type.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.